In [1]:
import optuna

In [2]:
study = optuna.create_study()

[I 2021-12-15 21:27:55,852] A new study created in memory with name: no-name-7d8bf915-a54e-4b0a-930c-74de44785004


In [3]:
study.optimize?

Signature:
study.optimize(
    func: Callable[[optuna.trial._trial.Trial], Union[float, Sequence[float]]],
    n_trials: Optional[int] = None,
    timeout: Optional[float] = None,
    n_jobs: int = 1,
    catch: Tuple[Type[Exception], ...] = (),
    callbacks: Optional[List[Callable[[ForwardRef('Study'), optuna.trial._frozen.FrozenTrial], NoneType]]] = None,
    gc_after_trial: bool = False,
    show_progress_bar: bool = False,
) -> None
Docstring:
Optimize an objective function.

Optimization is done by choosing a suitable set of hyperparameter values from a given
range. Uses a sampler which implements the task of value suggestion based on a specified
distribution. The sampler is specified in :func:`~optuna.study.create_study` and the
default choice for the sampler is TPE.
See also :class:`~optuna.samplers.TPESampler` for more details on 'TPE'.

Example:

    .. testcode::

        import optuna


        def objective(trial):
            x = trial.suggest_float("x", -1, 1)
          

In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from torchvision.models.detection import maskrcnn_resnet50_fpn
from dotenv import load_dotenv
from easydict import EasyDict
from tqdm import tqdm
import pickle

from src.postprocessing import postprocess_predictions
from src.dataset import CellDataset
from src.augmentations import train_transform
from src.utils import images2device
from src.iou_metric import iou_map

current_dir = Path(".")
load_dotenv()

True

In [2]:
experiment_name = 'first_baseline_with_metric'
weights_name = f'{experiment_name}.ckpt'
weights_path = Path(os.environ["weights_path"]) / weights_name

In [3]:
device = 'cuda:1'
config = EasyDict(
    dataset_path=Path(os.environ["dataset_path"]),
    device="cuda:1",
    val_size=0.2,
    batch_size=6,
    num_workers=30,
    epochs=1,
    mask_threshold=0.5,
    score_threshold=0.2,
    nms_threshold=None,
)

In [4]:
model = maskrcnn_resnet50_fpn(progress=False, num_classes=2)
model.load_state_dict(torch.load(weights_path, map_location=torch.device(device)))
model.to(device)
model.eval()
print()

In [5]:
train_dataloader = DataLoader(
    dataset=CellDataset(cfg=config, mode="train", transform=train_transform),
    num_workers=30,
    batch_size=config.batch_size,
    collate_fn=lambda x: tuple(zip(*x)),
)

In [8]:
def evaluate_thresholds(trial):
    mask_thr = trial.suggest_float('mask_thr', 0.0, 0.95)
    score_thr = trial.suggest_float('score_thr', 0.0, 0.95)
    nms_thr = trial.suggest_float('nms_thr', 0.0, 1.0)
    ious = []
    for batch_idx, (images, targets) in enumerate(tqdm(train_dataloader)):
        images= images2device(images, device)
        outputs = model(images)
        processed_outputs = postprocess_predictions(outputs, mask_thr, score_thr, nms_thr)
        for i in range(len(images)):
            predicted_masks = processed_outputs[i]['masks']
            true_masks = targets[i]['masks']
            # print(predicted_masks.shape, true_masks.shape)
            iou = iou_map(predicted_masks, true_masks.numpy())
            ious.append(iou)
    return np.mean(ious)

In [9]:
os.makedirs('thresholds', exist_ok=True)

In [11]:
import optuna
study = optuna.create_study(direction='maximize')
for i in range(5):
    study.optimize(evaluate_thresholds, n_trials=1)
    with open(f"thresholds/{experiment_name}.pickle", 'wb') as file:
        pickle.dump(study, file)
    study.trials_dataframe().to_csv(f"thresholds/{experiment_name}.csv", index=False)

[I 2021-12-15 21:21:06,529] A new study created in memory with name: no-name-717f2439-c48b-4aa9-ad42-ecc3e0e279cf


TypeError: optimize() got an unexpected keyword argument 'maximize'